In [625]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_8d1c53da1cbb4d8bb89ec5eaf6508b9e.csv"
df=pd.read_csv(url)
df.head()

,sex,age,Time,Number_of_Warts,Type,Area,Result_of_Treatment
0,1,35,12.00,5,1,100,0
1,1,29,7.00,5,1,96,1
2,1,50,8.00,1,3,132,0
3,1,32,11.75,7,3,750,0
4,1,67,9.25,1,1,42,0


In [626]:
df.shape

(90, 7)

In [627]:
df.isna().any()

sex                    False
age                    False
Time                   False
Number_of_Warts        False
Type                   False
Area                   False
Result_of_Treatment    False
dtype: bool

In [628]:
df.groupby("Result_of_Treatment").count().T

Result_of_Treatment,0,1
sex,42,48
age,42,48
Time,42,48
Number_of_Warts,42,48
Type,42,48
Area,42,48


A balanced dataset.

In [629]:
X=df.drop(["Result_of_Treatment"],axis=1)
X.head()

,sex,age,Time,Number_of_Warts,Type,Area
0,1,35,12.00,5,1,100
1,1,29,7.00,5,1,96
2,1,50,8.00,1,3,132
3,1,32,11.75,7,3,750
4,1,67,9.25,1,1,42


In [630]:
y=df[["Result_of_Treatment"]]
y.head()

,Result_of_Treatment
0,0
1,1
2,0
3,0
4,0


In [631]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X=pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,sex,age,Time,Number_of_Warts,Type,Area
0,-0.956501,0.481695,1.279144,-0.144085,-0.777778,0.108143
1,-0.956501,0.030106,-0.196791,-0.144085,-0.777778,0.077609
2,-0.956501,1.610667,0.098396,-1.271709,1.444444,0.352419
3,-0.956501,0.255900,1.205347,0.419727,1.444444,5.070003
4,-0.956501,2.890170,0.467380,-1.271709,-0.777778,-0.334607


In [632]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
print(X_train.shape)
print(X_test.shape)

(76, 6)
(14, 6)


In [633]:
from sklearn.ensemble import GradientBoostingClassifier
gradient=GradientBoostingClassifier(loss="deviance",learning_rate=0.005,n_estimators=250,subsample=0.95,criterion="mse",min_samples_split=3,min_samples_leaf=2,max_depth=3,max_features="auto",validation_fraction=0.15,random_state=0).fit(X_train,y_train)
gradient

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='mse', init=None, learning_rate=0.005,
              loss='deviance', max_depth=3, max_features='auto',
              max_leaf_nodes=None, min_impurity_decrease=0.0,
              min_impurity_split=None, min_samples_leaf=2,
              min_samples_split=3, min_weight_fraction_leaf=0.0,
              n_estimators=250, n_iter_no_change=None, presort='auto',
              random_state=0, subsample=0.95, tol=0.0001,
              validation_fraction=0.15, verbose=0, warm_start=False)

In [634]:
from sklearn.metrics import accuracy_score
y0_pred=gradient.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=gradient.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.9868421052631579
Accuracy for test data is:  0.9285714285714286


In [635]:
from sklearn.metrics import roc_auc_score
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.9878048780487805
AUC for test data is:  0.9285714285714286


In [636]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[35  0]
 [ 1 40]]
Confusion matrix for test data is: 
 [[7 0]
 [1 6]]


In [640]:
from sklearn.metrics import f1_score
print("F1 score for for train data is: ",f1_score(y_train,y0_pred))
print("F1 score for test data is: ", f1_score(y_test,y1_pred))

F1 score for for train data is:  0.9876543209876543
F1 score for test data is:  0.923076923076923


In [644]:
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(n_estimators=200,criterion="entropy",max_depth=4,min_samples_split=3,min_samples_leaf=1,max_features="auto",bootstrap=True,n_jobs=-1,warm_start=True,class_weight="balanced",random_state=0).fit(X_train,y_train)
forest

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:502: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or "balanced_subsample" are '


RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=4, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=3, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=-1, oob_score=False, random_state=0,
            verbose=0, warm_start=True)

In [645]:
y0_pred=forest.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=forest.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  1.0
Accuracy for test data is:  0.9285714285714286


In [646]:
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  1.0
AUC for test data is:  0.9285714285714286


In [647]:
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[35  0]
 [ 0 41]]
Confusion matrix for test data is: 
 [[7 0]
 [1 6]]


In [648]:
print("F1 score for for train data is: ",f1_score(y_train,y0_pred))
print("F1 score for test data is: ", f1_score(y_test,y1_pred))

F1 score for for train data is:  1.0
F1 score for test data is:  0.923076923076923


In [649]:
from sklearn import svm
svm=svm.SVC(C=0.7,kernel="rbf",degree=3,gamma="auto",cache_size=175,decision_function_shape="ovr",probability=True,random_state=0).fit(X_train,y_train)
svm

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=0.7, cache_size=175, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=0, shrinking=True, tol=0.001,
  verbose=False)

In [650]:
y0_pred=svm.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=svm.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.9473684210526315
Accuracy for test data is:  0.7857142857142857


In [651]:
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.9470383275261324
AUC for test data is:  0.7857142857142857


In [652]:
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[33  2]
 [ 2 39]]
Confusion matrix for test data is: 
 [[7 0]
 [3 4]]


In [653]:
print("F1 score for for train data is: ",f1_score(y_train,y0_pred))
print("F1 score for test data is: ", f1_score(y_test,y1_pred))

F1 score for for train data is:  0.9512195121951219
F1 score for test data is:  0.7272727272727273


In [83]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam

In [84]:
length=X_test.shape[1]
length

6

In [85]:
from keras.utils import to_categorical
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
num_classes=y_test.shape[1]
print(num_classes)

2


In [102]:
def classification_model():
    # create model
    model = Sequential()
    model.add(Dense(length, activation='relu', input_shape=(length,)))
    model.add(Dense(80, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(80, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(80, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(80, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    opt=Adam(lr=0.005)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [106]:
model=classification_model()
model.fit(X_train,y_train,validation_data=(X_test, y_test),epochs=50, verbose=2)
model.evaluate(X_test,y_test,verbose=0)

Train on 76 samples, validate on 14 samples
Epoch 1/50
 - 3s - loss: 0.6685 - acc: 0.5789 - val_loss: 0.6004 - val_acc: 0.5714
Epoch 2/50
 - 0s - loss: 0.5281 - acc: 0.7895 - val_loss: 0.5086 - val_acc: 0.7857
Epoch 3/50
 - 0s - loss: 0.4314 - acc: 0.7763 - val_loss: 0.4811 - val_acc: 0.7857
Epoch 4/50
 - 0s - loss: 0.2559 - acc: 0.8947 - val_loss: 0.5779 - val_acc: 0.7857
Epoch 5/50
 - 0s - loss: 0.2411 - acc: 0.8816 - val_loss: 0.7113 - val_acc: 0.7857
Epoch 6/50
 - 0s - loss: 0.1502 - acc: 0.9474 - val_loss: 0.8634 - val_acc: 0.7857
Epoch 7/50
 - 0s - loss: 0.1822 - acc: 0.8947 - val_loss: 0.9873 - val_acc: 0.7857
Epoch 8/50
 - 0s - loss: 0.1476 - acc: 0.9079 - val_loss: 1.0697 - val_acc: 0.7857
Epoch 9/50
 - 0s - loss: 0.2041 - acc: 0.9342 - val_loss: 1.1093 - val_acc: 0.7857
Epoch 10/50
 - 0s - loss: 0.1230 - acc: 0.9737 - val_loss: 1.1872 - val_acc: 0.7857
Epoch 11/50
 - 0s - loss: 0.1105 - acc: 0.9474 - val_loss: 1.2872 - val_acc: 0.7857
Epoch 12/50
 - 0s - loss: 0.1410 - acc: 0

[2.7996253967285156, 0.7857142686843872]

Neural network does not perform very well when data is limited. For this small dataset that has only 90 instances, GradientBoostingClassifier, RandomForestClassifier and SVM all outperform the Neural Network model.

Both the GradientBoostingClassifer model and the RandomForestClassifer model delivered 92.86% accuracy on test data. Both model correctly classified all instances of Class 0 and misclassified one instance of Class 1. Can we perfect these two models by lowering the threshold?

Run the lines for the other models again, starting from the line that splits the dataset ito train and test subsets.

In [619]:
y_prob=svm.predict_proba(X_test)
y_probs1=y_prob[:,1]

In [620]:
import numpy as np
thresholds=np.arange(0,1,0.001)

In [621]:
def to_labels(pos_probs,threshold):
    return (pos_probs >= threshold).astype('int')

In [622]:
from sklearn.metrics import f1_score
scores=[f1_score(y_test, to_labels(y_probs1,t)) for t in thresholds]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [623]:
ix=np.argmax(scores)
print((thresholds[ix],scores[ix]))

(0.481, 0.7272727272727273)


In [624]:
y_probs1=pd.DataFrame(y_probs1).rename(columns={0:"Pred_1"})
pred_1=y_probs1[y_probs1["Pred_1"]>=0.481]
pred_1

,Pred_1
1,0.980770
5,0.957615
12,0.984286
13,0.991341


With the default threshold, the SVM model correctly classified 4 of the positive cases in the test data. After lowering the threshold, the SVM model still just classified 4 positive cases. 

Let's see if adjusting the threshold will improve the performance of GradientBoostingClassifier and RandomForestClassifer or not.

In [57]:
y_prob=gradient.predict_proba(X_test)

In [58]:
y_probs1=y_prob[:,1]

In [59]:
import numpy as np
thresholds=np.arange(0,1,0.001)

In [60]:
def to_labels(pos_probs,threshold):
    return (pos_probs >= threshold).astype('int')

In [61]:
from sklearn.metrics import f1_score
scores=[f1_score(y_test, to_labels(y_probs1,t)) for t in thresholds]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [62]:
ix=np.argmax(scores)
print((thresholds[ix],scores[ix]))

(0.467, 0.923076923076923)


In [63]:
y_probs1=pd.DataFrame(y_probs1).rename(columns={0:"Pred_1"})
pred_1=y_probs1[y_probs1["Pred_1"]>=0.467]
pred_1

,Pred_1
1,0.863183
5,0.867593
7,0.842028
10,0.800138
12,0.867593
13,0.867593


In [64]:
y_test=y_test.reset_index(drop=True)
y_test

,Result_of_Treatment
0,0
1,1
2,0
3,0
4,0
5,1
6,0
7,1
8,0
9,0


In [65]:
test_1=y_test[y_test["Result_of_Treatment"]==1]
test_1

,Result_of_Treatment
1,1
5,1
7,1
10,1
11,1
12,1
13,1


In [66]:
print("Number of actual positive cases :",len(test_1))

Number of actual positive cases : 7


In [67]:
print("Number of predicted positive cases: ", len(pred_1))

Number of predicted positive cases:  6


Lowering the threshold did not allow GradientBoostingClassifier the model to detect the only case that it missed. Let's see which instance it missed.

In [68]:
test_1.index

Int64Index([1, 5, 7, 10, 11, 12, 13], dtype='int64')

In [69]:
pred_1.index

Int64Index([1, 5, 7, 10, 12, 13], dtype='int64')

So, the mode missed instance 11. Let's see the predicted probability for instance 11.

In [73]:
y_probs1.loc[11]

Pred_1    0.153563
Name: 11, dtype: float64

The predicted probability for instance 11 is 0.153563. So, lowering the threshold to the optimal threshold 0.467 did not help the model detect this positive instance.

What about the RandomForestClassifier model? Would lowering the threshold allow the RandomForestClassifier model detect the only positive instance that it missed?

In [77]:
y_prob=forest.predict_proba(X_test)

In [78]:
y_probs1=y_prob[:,1]

In [79]:
import numpy as np
thresholds=np.arange(0,1,0.001)

In [80]:
from sklearn.metrics import f1_score
scores=[f1_score(y_test, to_labels(y_probs1,t)) for t in thresholds]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [81]:
ix=np.argmax(scores)
print((thresholds[ix],scores[ix]))

(0.459, 0.923076923076923)


In [83]:
y_probs1=pd.DataFrame(y_probs1).rename(columns={0:"Pred_1"})
pred_1=y_probs1[y_probs1["Pred_1"]>=0.459]
pred_1

,Pred_1
1,0.915569
5,0.986587
7,0.523366
10,0.607031
12,0.968208
13,0.975803


Lowering the threshold did not allow the RandomForestClassifier model to detect instance 11 either.

In [654]:
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_6d1f4a8188164697b8414877a17e5adf.csv"
df=pd.read_csv(url)
df.head()

,sex,age,Time,Number_of_Warts,Type,Area,induration_diameter,Result_of_Treatment
0,1,22,2.25,14,3,51,50,1
1,1,15,3.00,2,3,900,70,1
2,1,16,10.50,2,1,100,25,1
3,1,27,4.50,9,3,80,30,1
4,1,20,8.00,6,1,45,8,1


In [655]:
df.shape

(90, 8)

In [656]:
df.isna().any()

sex                    False
age                    False
Time                   False
Number_of_Warts        False
Type                   False
Area                   False
induration_diameter    False
Result_of_Treatment    False
dtype: bool

In [657]:
df.groupby("Result_of_Treatment").count().T

Result_of_Treatment,0,1
sex,19,71
age,19,71
Time,19,71
Number_of_Warts,19,71
Type,19,71
Area,19,71
induration_diameter,19,71


For immunotherapy dataset, there is a clear class imbalance here.

In [658]:
X=df.drop(["Result_of_Treatment"],axis=1)
X.head()

,sex,age,Time,Number_of_Warts,Type,Area,induration_diameter
0,1,22,2.25,14,3,51,50
1,1,15,3.00,2,3,900,70
2,1,16,10.50,2,1,100,25
3,1,27,4.50,9,3,80,30
4,1,20,8.00,6,1,45,8


In [659]:
y=df[["Result_of_Treatment"]]
y.head()

,Result_of_Treatment
0,1
1,1
2,1
3,1
4,1


In [660]:
scaler=StandardScaler()
X=pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,sex,age,Time,Number_of_Warts,Type,Area,induration_diameter
0,-1.093216,-0.743342,-1.616588,1.875384,1.572168,-0.329031,2.083116
1,-1.093216,-1.318656,-1.373153,-0.989418,1.572168,5.920346,3.251218
2,-1.093216,-1.236468,1.061196,-0.989418,-0.867403,0.031652,0.622988
3,-1.093216,-0.332404,-0.886283,0.681717,1.572168,-0.115566,0.915014
4,-1.093216,-0.907718,0.249746,-0.034484,-0.867403,-0.373196,-0.369899


In [661]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
print(X_train.shape)
print(X_test.shape)

(76, 7)
(14, 7)


In [662]:
gradient=GradientBoostingClassifier(loss="exponential",learning_rate=0.01,n_estimators=200,subsample=0.95,criterion="mse",min_samples_split=3,min_samples_leaf=1,max_depth=3,max_features="log2",validation_fraction=0.15,random_state=0).fit(X_train,y_train)
gradient

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='mse', init=None, learning_rate=0.01,
              loss='exponential', max_depth=3, max_features='log2',
              max_leaf_nodes=None, min_impurity_decrease=0.0,
              min_impurity_split=None, min_samples_leaf=1,
              min_samples_split=3, min_weight_fraction_leaf=0.0,
              n_estimators=200, n_iter_no_change=None, presort='auto',
              random_state=0, subsample=0.95, tol=0.0001,
              validation_fraction=0.15, verbose=0, warm_start=False)

In [663]:
y0_pred=gradient.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=gradient.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.9342105263157895
Accuracy for test data is:  0.8571428571428571


In [664]:
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.84375
AUC for test data is:  0.6666666666666667


In [665]:
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[11  5]
 [ 0 60]]
Confusion matrix for test data is: 
 [[ 1  2]
 [ 0 11]]


The model correctly classified all positive cases, but misclassified 2 negative cases as positive.

In [666]:
y_prob=gradient.predict_proba(X_test)

Since the model misclassified the negative cases, we want the predicted probabilities of the negative class.

In [667]:
y_probs0=y_prob[:,0]

In [668]:
import numpy as np
thresholds=np.arange(0,1,0.001)

In [669]:
from sklearn.metrics import f1_score
scores=[f1_score(y_test, to_labels(y_probs0,t)) for t in thresholds]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [670]:
ix=np.argmax(scores)
print((thresholds[ix],scores[ix]))

(0.0, 0.88)


The optimal threshold is 0. However, if we set the threshold to be 0, all instances in the test data would be classified as negative. We certainly do not want that to happen. So, lowering the threshold does not work in this case.

In [671]:
forest=RandomForestClassifier(n_estimators=100,criterion="gini",max_depth=2,min_samples_split=2,min_samples_leaf=2,max_features="log2",bootstrap=True,n_jobs=-1,warm_start=True,class_weight="balanced_subsample",random_state=0).fit(X_train,y_train)
forest

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:502: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or "balanced_subsample" are '


RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='gini', max_depth=2, max_features='log2',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=2,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False, random_state=0,
            verbose=0, warm_start=True)

In [672]:
y0_pred=forest.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=forest.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.9210526315789473
Accuracy for test data is:  0.8571428571428571


In [673]:
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.8354166666666667
AUC for test data is:  0.6666666666666667


In [674]:
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[11  5]
 [ 1 59]]
Confusion matrix for test data is: 
 [[ 1  2]
 [ 0 11]]


The RandomForestClassifier model delviered the same result as the GradientBoostingClassifier model.

In [675]:
from sklearn import svm
svm=svm.SVC(C=0.8,kernel="rbf",degree=2,gamma="scale",cache_size=150,decision_function_shape="ovo",probability=True,random_state=0).fit(X_train,y_train)
svm

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=0.8, cache_size=150, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=2, gamma='scale', kernel='rbf',
  max_iter=-1, probability=True, random_state=0, shrinking=True, tol=0.001,
  verbose=False)

In [676]:
y0_pred=svm.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=svm.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.8421052631578947
Accuracy for test data is:  0.7857142857142857


In [677]:
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.625
AUC for test data is:  0.5


In [678]:
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[ 4 12]
 [ 0 60]]
Confusion matrix for test data is: 
 [[ 0  3]
 [ 0 11]]


The Support Vector Machine (SVM) model misclassified all negative cases as positive.

In [679]:
y_probs=svm.predict_proba(X_test)
y_probs0=y_probs[:,0]

In [680]:
import numpy as np
thresholds=np.arange(0,1,0.001)

In [681]:
from sklearn.metrics import f1_score
scores=[f1_score(y_test, to_labels(y_probs0,t)) for t in thresholds]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [682]:
ix=np.argmax(scores)
print((thresholds[ix],scores[ix]))

(0.0, 0.88)


There is no way to improve the SVM moodel through adjusting the threshold either.

Lowering the threshold did not improve model performance. Let's try oversampling the minority class in train data.

In [683]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
print(X_train.shape)
print(X_test.shape)

(76, 7)
(14, 7)


In [684]:
df_t=pd.concat([X_train,y_train],axis=1)
df_t0=df_t[df_t["Result_of_Treatment"]==0]
df_t0=pd.concat([df_t0]*2,axis=0)
df_t1=df_t[df_t["Result_of_Treatment"]==1]
df_t=pd.concat([df_t0,df_t1],axis=0)
df_t.shape

(92, 8)

In [685]:
X_train=df_t.drop(["Result_of_Treatment"],axis=1)
X_train.shape

(92, 7)

In [686]:
y_train=df_t[["Result_of_Treatment"]]
y_train.shape

(92, 1)

In [687]:
gradient=GradientBoostingClassifier(loss="exponential",learning_rate=0.05,n_estimators=200,subsample=1,criterion="mse",min_samples_split=3,min_samples_leaf=2,max_depth=4,max_features="auto",validation_fraction=0.15,random_state=0).fit(X_train,y_train)
gradient

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='mse', init=None, learning_rate=0.05,
              loss='exponential', max_depth=4, max_features='auto',
              max_leaf_nodes=None, min_impurity_decrease=0.0,
              min_impurity_split=None, min_samples_leaf=2,
              min_samples_split=3, min_weight_fraction_leaf=0.0,
              n_estimators=200, n_iter_no_change=None, presort='auto',
              random_state=0, subsample=1, tol=0.0001,
              validation_fraction=0.15, verbose=0, warm_start=False)

In [688]:
y0_pred=forest.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=forest.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.8804347826086957
Accuracy for test data is:  0.8571428571428571


In [689]:
forest=RandomForestClassifier(n_estimators=150,criterion="entropy",max_depth=5,min_samples_split=2,min_samples_leaf=1,max_features="sqrt",bootstrap=False,n_jobs=-1,warm_start=False,class_weight="balanced",random_state=0).fit(X_train,y_train)
forest

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=False, class_weight='balanced',
            criterion='entropy', max_depth=5, max_features='sqrt',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=-1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [690]:
y0_pred=forest.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=forest.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  1.0
Accuracy for test data is:  0.8571428571428571


Neither the adjusting the threshold nor oversamping the minority class in train data improved model performance. Let's try a different classification algorithm.

In [691]:
from sklearn import svm
svm=svm.SVC(C=0.8,kernel="sigmoid",degree=5,gamma="auto",cache_size=100,decision_function_shape="ovr",probability=True,random_state=0).fit(X_train,y_train)
svm

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=0.8, cache_size=100, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=5, gamma='auto', kernel='sigmoid',
  max_iter=-1, probability=True, random_state=0, shrinking=True, tol=0.001,
  verbose=False)

In [692]:
y0_pred=svm.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=svm.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.7282608695652174
Accuracy for test data is:  0.9285714285714286


In [693]:
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.6604166666666667
AUC for test data is:  0.8333333333333334


In [694]:
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[14 18]
 [ 7 53]]
Confusion matrix for test data is: 
 [[ 2  1]
 [ 0 11]]


In [695]:
print("F1 score for for train data is: ",f1_score(y_train,y0_pred))
print("F1 score for test data is: ", f1_score(y_test,y1_pred))

F1 score for for train data is:  0.8091603053435115
F1 score for test data is:  0.9565217391304348


There was no way to improve any of the models by adjusting the threshold. Oversampling the minority class in the train data did not improve the GradientBoostingClassifier model and the RandomForestClassifier model either. However, oversampling the minority class in the train data dramatically improve the SVM model. AUC for test data increased from 0.5 to 0.8333. With oversampling, the SVM model went from misclassifying all negative classes as positive class to misclassifying one one negative class as positive class.